# Testing the created framework

In [ ]:
from utils import *
from pytorchNet import MLP, train_pytorch_model
from MSELoss import MSELoss
import torch
import seaborn as sns
import glob
import cv2
%load_ext autoreload
%autoreload 2

## Load dataset

In [ ]:
train_set, train_target = generate_disc_set(2500)
test_set, test_target = generate_disc_set(2500)

## Create a model & test forward

In [ ]:
model = create_model()
output = model.forward(train_set,no_grad=True)

## Test the backward and no_grad

### No_grad

In [ ]:
criterion = MSELoss()
output = model.forward(train_set,no_grad=True)
loss = criterion.forward(output,train_target)
grdwrtoutput = criterion.backward()

In [ ]:
model.backward(grdwrtoutput)

### backward with grad

In [ ]:
output = model.forward(train_set,no_grad=False)
loss = criterion.forward(output,train_target)
grdwrtoutput = criterion.backward()

In [ ]:
model.backward(grdwrtoutput)

### Check the shapes of the gradients 

In [ ]:
for module in model.sequence:
    if len(module.params) != 0:
        print(module.params[1].shape)

# Try to train a model 

In [ ]:
torch.manual_seed(100)
model = create_model()
train_model(model,train_set,train_target,epochs=200)

In [ ]:
accuracy_test = compute_nb_errors(model,test_set,test_target)
accuracy_train = compute_nb_errors(model,train_set,train_target)

In [ ]:
print("accuracy on train set = ",round(accuracy_train,1))
print("accuracy on test set = ",round(accuracy_test,1))

## Comparison with Pytorch

In [ ]:
torch.manual_seed(100)
model_torch = MLP()
train_pytorch_model(model_torch,train_set,train_target,epochs=200)

In [ ]:
accuracy_test = compute_nb_errors(model_torch,test_set,test_target,pytorch=True)
accuracy_train = compute_nb_errors(model_torch,train_set,train_target,pytorch=True)

In [ ]:
print("accuracy on train set = ",round(accuracy_train,1))
print("accuracy on test set = ",round(accuracy_test,1))

# Visualization

In [ ]:
X = torch.linspace(0,1,1000)
Y = torch.linspace(0,1,1000)
grid_x, grid_y = torch.meshgrid(X,Y)
grid_x_vector = grid_x.reshape(-1,1)
grid_y_vector = grid_y.reshape(-1,1)
inputs = torch.cat((grid_x_vector,grid_y_vector),dim=1)
predicted = model.forward(inputs,no_grad=True)
predicted = predicted.reshape(grid_x.shape[0],-1)
with torch.no_grad():
    predicted_torch = model_torch(inputs)
    predicted_torch = predicted_torch.reshape(grid_x.shape[0],-1)

In [ ]:
fig = plt.figure(figsize=[16,7])
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)
cs1 = ax1.contourf(grid_x,grid_y,predicted)
cs2 = ax2.contourf(grid_x,grid_y,predicted_torch)
ax1.set_title("Our framework")
ax2.set_title("Pytorch")
plt.show()

# GIF

In [ ]:
model = create_model()
model_torch = MLP()
steps=3
epochs=90
generate_images(train_set,train_target,model,model_torch,steps,epochs,folder="figures")

In [ ]:
path_img = ["figures/epochs" + str(nb_epochs) + ".jpg" for nb_epochs in range(steps,epochs+1,steps)]
img_arr = []
for filename in path_img:
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_arr.append(img)
 

out = cv2.VideoWriter('mygif.avi',cv2.VideoWriter_fourcc(*'DIVX'), 3, size)
 
for i in range(len(img_arr)):
    out.write(img_arr[i])
out.release()